# Imports

In [2]:
import matplotlib.pyplot as plt
import random
import os
import numpy as np
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import utils, optimizers
from PIL import Image
import shutil
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras.applications import densenet
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
from tensorflow import convert_to_tensor
import matplotlib.image as mpimg
import pickle

2023-03-13 15:06:23.127871: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Create DataFrame

In [3]:
#These variables can be changes, excluding train_dir
train_dir = "../raw_data/Training"
img_height, img_width = 256, 256
batch_size = 32
poses = sorted(os.listdir("../raw_data/Training"))

In [4]:
poses

['.ipynb_checkpoints',
 'Boat_Pose_or_Paripurna_Navasana_',
 'Bound_Angle_Pose',
 'Bow_Pose_or_Dhanurasana_',
 'Bridge_Pose_or_Setu_Bandha_Sarvangasana_',
 'Camel_Pose_or_Ustrasana_',
 'Cat_Cow_Pose_or_Marjaryasana_',
 'Cat_Pose',
 'Chair_Pose_or_Utkatasana_',
 'Child_Pose',
 'Cobra_Pose_or_Bhujangasana_',
 'Cockerel_Pose',
 'Corpse_Pose_or_Savasana_',
 'Cow_Face_Pose_or_Gomukhasana_',
 'Cow_Pose',
 'Crane_(Crow)_Pose',
 'Downward-Facing_Dog_pose_or_Adho_Mukha_Svanasana_',
 'Eagle_Pose_or_Garudasana_',
 'Eight-Angle_Pose_or_Astavakrasana_',
 'Extended_Puppy_Pose_or_Uttana_Shishosana_',
 'Extended_Revolved_Side_Angle_Pose_or_Utthita_Parsvakonasana_Twist',
 'Extended_Revolved_Side_Angle_Pose_or_Utthita_Parsvakonasana_Untwist',
 'Extended_Revolved_Triangle_Pose_or_Utthita_Trikonasana_',
 'Feathered_Peacock_Pose_or_Pincha_Mayurasana_',
 'Firefly_Pose_or_Tittibhasana_',
 'Fish_Pose_or_Matsyasana_',
 'Four-Limbed_Staff',
 'Frog_Pose_or_Bhekasana',
 'Garland_Pose_or_Malasana_',
 'Gate_Pose_or

In [5]:
#Splits into train_generator and validation_generator
#This bulk uploads the images
#Creates target (y) for us!

#Play around with the interpolation argument - bicubic, lanczos??? 

train_datagen = ImageDataGenerator(rescale=1./255,
                                    vertical_flip=True,
                                    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
                                    train_dir,
                                    target_size=(img_height, img_width),
                                    batch_size=batch_size,
                                    class_mode='categorical',
                                    subset='training',
                                    keep_aspect_ratio=True,
                                    interpolation='lanczos') # set as training data

validation_generator = train_datagen.flow_from_directory(
                                    train_dir, # same directory as training data
                                    target_size=(img_height, img_width),
                                    batch_size=batch_size,
                                    class_mode='categorical',
                                    subset='validation',
                                    keep_aspect_ratio=True,
                                    interpolation='lanczos') # set as validation data

Found 11510 images belonging to 83 classes.
Found 2837 images belonging to 83 classes.


# Model

In [6]:
...
# Model needs building + transfer learning  
...
def initialize_model():
    base_model = densenet.DenseNet169(
                        include_top=False,
                        weights='imagenet',
                        input_shape=(img_height, img_width, 3),
                        classifier_activation='softmax')
    
    base_model.trainable = True

    model = models.Sequential([ 
        base_model,
        layers.Flatten(),
        layers.Dropout(0.4),
        layers.Dense(1000,activation="relu"),
        layers.Dropout(0.3),
        layers.Dense(900, activation="relu"),
        layers.Dropout(0.3),
        layers.Dense(800, activation="relu"),
        layers.Dropout(0.3),
        layers.Dense(700, activation="relu"),
        layers.Dense(83, activation="softmax")
    ])
    
    
    opt = optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=opt,
                loss='categorical_crossentropy',
                metrics=['accuracy']) 
    
    return model

In [7]:
model = initialize_model()

2023-03-13 15:06:48.464958: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-13 15:06:50.260589: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-13 15:06:50.262723: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-13 15:06:50.285159: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operation

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet169 (Functional)    (None, 8, 8, 1664)        12642880  
                                                                 
 flatten (Flatten)           (None, 106496)            0         
                                                                 
 dropout (Dropout)           (None, 106496)            0         
                                                                 
 dense (Dense)               (None, 1000)              106497000 
                                                                 
 dropout_1 (Dropout)         (None, 1000)              0         
                                                                 
 dense_1 (Dense)             (None, 900)               900900    
                                                                 
 dropout_2 (Dropout)         (None, 900)               0

In [9]:
es = EarlyStopping(patience=5, restore_best_weights=True)

In [10]:
#fit model - fit on train_generator (both X and y) and the validation data is validation_generator
history = model.fit(
                train_generator,
                steps_per_epoch = train_generator.samples // batch_size,
                validation_data = validation_generator, 
                validation_steps = validation_generator.samples // batch_size,
                epochs=500,
                callbacks=[es],
                batch_size=batch_size)

Epoch 1/500


2023-03-13 15:07:57.218402: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401
2023-03-13 15:08:13.688554: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x55b4fd5e76c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-03-13 15:08:13.688607: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2023-03-13 15:08:14.220284: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-03-13 15:08:15.547364: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


  4/359 [..............................] - ETA: 5:45 - loss: 5.1488 - accuracy: 0.0312    

/opt/conda/envs/python3106/lib/python3.10/site-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


359/359 [==============================] - ETA: 0s - loss: 4.2962 - accuracy: 0.0473

/opt/conda/envs/python3106/lib/python3.10/site-packages/PIL/TiffImagePlugin.py:858: UserWarning: Truncated File Read
  warnings.warn(str(msg))


359/359 [==============================] - 700s 2s/step - loss: 4.2962 - accuracy: 0.0473 - val_loss: 3.7282 - val_accuracy: 0.1026
Epoch 2/500
359/359 [==============================] - 299s 832ms/step - loss: 3.4490 - accuracy: 0.1396 - val_loss: 2.8877 - val_accuracy: 0.2447
Epoch 3/500
359/359 [==============================] - 300s 835ms/step - loss: 2.6795 - accuracy: 0.2705 - val_loss: 2.1154 - val_accuracy: 0.4226
Epoch 4/500
359/359 [==============================] - 299s 832ms/step - loss: 2.1334 - accuracy: 0.3986 - val_loss: 1.7943 - val_accuracy: 0.4925
Epoch 5/500
359/359 [==============================] - 300s 835ms/step - loss: 1.7428 - accuracy: 0.4916 - val_loss: 1.6124 - val_accuracy: 0.5550
Epoch 6/500
359/359 [==============================] - 299s 833ms/step - loss: 1.5179 - accuracy: 0.5519 - val_loss: 1.4188 - val_accuracy: 0.6048
Epoch 7/500
359/359 [==============================] - 299s 832ms/step - loss: 1.3277 - accuracy: 0.6036 - val_loss: 1.3085 - val_acc

In [10]:
history.history

NameError: name 'history' is not defined

In [ ]:
def plot_history(history, title='', axs=None, exp_name=""):
    if axs is not None:
        ax1, ax2 = axs
    else:
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    if len(exp_name) > 0 and exp_name[0] != '_':
        exp_name = '_' + exp_name
    ax1.plot(history.history['loss'], label='train' + exp_name)
    ax1.plot(history.history['val_loss'], label='val' + exp_name)
    #ax1.set_ylim(0., 2.2)
    ax1.set_title('loss')
    ax1.legend()

    ax2.plot(history.history['accuracy'], label='train accuracy'  + exp_name)
    ax2.plot(history.history['val_accuracy'], label='val accuracy'  + exp_name)
    #ax2.set_ylim(0.25, 1.)
    ax2.set_title('Accuracy')
    ax2.legend()
    return (ax1, ax2)

In [20]:
model.save_weights('model_weights')

In [ ]:
plot_history(history)

In [19]:
model.save("model_final.h5", save_format = "h5")

In [13]:
from tensorflow.keras.models import load_model

In [14]:
model1 = load_model("model1")

In [16]:
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet169 (Functional)    (None, 8, 8, 1664)        12642880  
                                                                 
 flatten (Flatten)           (None, 106496)            0         
                                                                 
 dropout (Dropout)           (None, 106496)            0         
                                                                 
 dense (Dense)               (None, 1000)              106497000 
                                                                 
 dropout_1 (Dropout)         (None, 1000)              0         
                                                                 
 dense_1 (Dense)             (None, 900)               900900    
                                                                 
 dropout_2 (Dropout)         (None, 900)               0

# Prediction

In [ ]:
test_dir = "../test"

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255) # set validation split

test_generator = test_datagen.flow_from_directory(
                                    test_dir,
                                    target_size=(img_height, img_width),
                                    batch_size=batch_size,
                                    class_mode='categorical',
                                    subset='training',
                                    keep_aspect_ratio=True,
                                    interpolation='bicubic') # set as training data

In [ ]:
plt.imshow(mpimg.imread("../test/test/IMG_5553.jpg"))

In [ ]:
prediction = poses[np.argmax(model.predict(test_generator))]
prediction

In [ ]:
prediction = poses[np.argmax(model.predict(test_generator))]
photo_path = f"../ground_truth/{prediction}.jpeg"
fig = plt.imshow(mpimg.imread(photo_path))
plt.title(prediction)
fig.axes.get_xaxis().set_visible(False)
fig.axes.get_yaxis().set_visible(False)

# Test

In [ ]:
test_dir = "../raw_data/Testing"

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255) # set validation split

test_generator = test_datagen.flow_from_directory(
                                    test_dir,
                                    target_size=(img_height, img_width),
                                    batch_size=batch_size,
                                    class_mode='categorical',
                                    subset='training',
                                    keep_aspect_ratio=True,
                                    interpolation='bicubic') # set as training data

In [ ]:
model.evaluate(test_generator)

# Probabilities

In [ ]:
df = pd.DataFrame(poses)
df["probability"] = model.predict(test_generator)[0]

In [ ]:
df[df[0]=="Sitting_pose_1_(normal)"]

# Save models

In [ ]:
with open("../saved_models/model_4.pkl", "wb") as file:
    pickle.dump(model, file)



In [ ]:
model = pickle.load(open("../saved_models/model_1.pkl","rb"))